# Get Partial Charges
Evaluate the oxidation states given the charge densities computed from DFT.

In [1]:
from co2rr.bader import compute_partial_charges
from co2rr.cp2k import pp_val
from ase.db import connect
from ase.io import read
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import numpy as np
import json

## Get Map of Composition to A/B Element(s)
We are going to summarize the oxidation states of the structure for the A and B sites. So, let's get a table of name -> A / B atoms

In [2]:
name_to_sites = {}
with connect('atoms-relax.db') as db:
    for row in db.select(''):
        # Get the identities for the host structure
        record = dict(
            a_sites=[row.key_value_pairs['a']],
            b_sites=[row.key_value_pairs['b']]
        )

        # Add the dopant if available
        if 'dopant' in row.key_value_pairs:
            dopant = row.key_value_pairs['dopant']
            if row.key_value_pairs['site'] == 'a':
                record['a_sites'].append(dopant)
            else:
                record['b_sites'].append(dopant)

        name=row.key_value_pairs['name']
        name_to_sites[name] = record

## Run Charge Analysis for All
Find the cube files and run chargemol for each

In [3]:
ox_data = []
for path in tqdm(Path('atoms-relax/').rglob('*.cube.gz')):
    run_type = path.name.split(".")[0]
    charges_path = path.parent / f'{run_type}.charges.json'

    # Run if needed
    if not charges_path.exists():
        charges = compute_partial_charges(path)
        charges_path.write_text(json.dumps(charges.tolist()))
    else:
        charges = json.loads(charges_path.read_text())

    # Load the structure then compute the charge
    atoms = read(path.with_name('relax.traj'), -1)
    oxi = [pp_val[e] - c for c, e in zip(charges, atoms.symbols)]

    # Summarize the oxidation state for each site
    name = path.parts[1]
    sites = name_to_sites[name]
    record = {'name': name}
    for tag, elems in [('A', sites['a_sites']), ('B', sites['b_sites']), ('O', ['O'])]:
        site_charges = [o for o, e in zip(oxi, atoms.symbols) if e in elems]
        record[tag] = np.mean(site_charges)
    ox_data.append(record)
ox_data = pd.DataFrame(ox_data)

130it [01:01,  2.11it/s]


In [4]:
ox_data.to_csv('oxidation-states.csv', index=False)